# Capítulo 03 - Importação e Exportação de CSV

Este notebook explora todas as formas de trabalhar com arquivos CSV no DuckDB, desde leitura simples até operações avançadas com configurações personalizadas.

## 📚 Tópicos Abordados:
1. Instalação e Preparação
2. Criando Dados de Exemplo
3. Leitura Básica de CSV
4. Leitura com Query SQL Direta
5. Conversão para DataFrame (Pandas)
6. Exportação para CSV
7. Configurações de Leitura (Delimitadores, Headers)
8. Lidando com Arquivos Grandes
9. Compressão (CSV.GZ)
10. Múltiplos Arquivos (Glob Patterns)
11. Criação de Tabelas Persistentes
12. Performance e Boas Práticas

## 1. Instalação e Preparação

Instalando dependências necessárias.

In [ ]:
# Instalar DuckDB e Pandas
%pip install duckdb pandas -q
print("✓ Pacotes instalados!")

In [ ]:
import duckdb
import pandas as pd
import os

print(f"DuckDB versão: {duckdb.__version__}")
print(f"Pandas versão: {pd.__version__}")
print("\n✓ Imports realizados com sucesso!")

## 2. Criando Dados de Exemplo

Vamos criar alguns arquivos CSV de exemplo para trabalhar.

In [ ]:
# Criar arquivo CSV simples
csv_content = """id,nome,idade,cidade,salario
1,Alice,28,São Paulo,5500.00
2,Bob,35,Rio de Janeiro,7200.00
3,Charlie,42,Belo Horizonte,6800.00
4,Diana,31,Curitiba,5900.00
5,Eduardo,29,Porto Alegre,6100.00
6,Fernanda,38,Salvador,7500.00
7,Gabriel,45,Brasília,8200.00
8,Helena,33,Fortaleza,6300.00
"""

with open("example.csv", "w", encoding="utf-8") as f:
    f.write(csv_content)

print("✓ Arquivo 'example.csv' criado!")
print(f"Tamanho: {os.path.getsize('example.csv')} bytes")

In [ ]:
# Criar arquivo CSV com delimitador diferente (ponto e vírgula)
csv_semicolon = """produto;categoria;preco;estoque
Notebook;Eletrônicos;3500.00;15
Mouse;Eletrônicos;50.00;120
Teclado;Eletrônicos;150.00;80
Cadeira;Móveis;899.00;25
Mesa;Móveis;1200.00;10
"""

with open("produtos.csv", "w", encoding="utf-8") as f:
    f.write(csv_semicolon)

print("✓ Arquivo 'produtos.csv' criado (delimitador: ';')!")

## 3. Leitura Básica de CSV

DuckDB oferece formas simples de ler arquivos CSV.

### 3.1 Usando `read_csv()`

In [ ]:
import duckdb

# Ler CSV como Relation (objeto DuckDB)
rel = duckdb.read_csv("example.csv")

print("Tipo do objeto:", type(rel))
print("\nPrimeiras linhas:")
rel.show()

print("\n✓ CSV lido com sucesso!")

### 3.2 Inspecionar Schema

In [ ]:
# Ver estrutura do CSV
print("Schema detectado:")
print(rel)

# Ou usando SQL
print("\nUsando DESCRIBE:")
duckdb.sql("DESCRIBE SELECT * FROM 'example.csv'").show()

## 4. Leitura com Query SQL Direta

DuckDB permite consultar CSV diretamente com SQL!

### 4.1 Query Simples

In [ ]:
# Query direta no arquivo CSV
print("Consulta SQL direta no CSV:")
result = duckdb.sql("SELECT * FROM 'example.csv'")
result.show()

print("\n✓ Query executada diretamente no arquivo!")

### 4.2 Queries com Filtros

In [ ]:
# Filtrar dados
print("Pessoas com idade > 35:")
duckdb.sql("""
    SELECT nome, idade, cidade, salario
    FROM 'example.csv'
    WHERE idade > 35
    ORDER BY idade DESC
""").show()

print("\nSalários acima de 7000:")
duckdb.sql("""
    SELECT nome, cidade, salario
    FROM 'example.csv'
    WHERE salario >= 7000
""").show()

### 4.3 Agregações

In [ ]:
# Estatísticas
print("Estatísticas gerais:")
duckdb.sql("""
    SELECT 
        COUNT(*) AS total_pessoas,
        AVG(idade) AS idade_media,
        AVG(salario) AS salario_medio,
        MAX(salario) AS maior_salario,
        MIN(salario) AS menor_salario
    FROM 'example.csv'
""").show()

print("\nPessoas por cidade:")
duckdb.sql("""
    SELECT 
        cidade,
        COUNT(*) AS quantidade,
        AVG(salario) AS salario_medio
    FROM 'example.csv'
    GROUP BY cidade
    ORDER BY salario_medio DESC
""").show()

## 5. Conversão para DataFrame (Pandas)

Integração perfeita com Pandas para análise de dados.

### 5.1 Converter para Pandas

In [ ]:
# Método 1: Via read_csv + df()
df1 = duckdb.read_csv("example.csv").df()

print("DataFrame via read_csv:")
print(df1)
print(f"\nTipo: {type(df1)}")
print(f"Shape: {df1.shape}")

In [ ]:
# Método 2: Via SQL + df()
df2 = duckdb.sql("SELECT * FROM 'example.csv' WHERE idade > 30").df()

print("DataFrame via SQL (com filtro):")
print(df2)
print(f"\nShape: {df2.shape}")

### 5.2 Análise com Pandas

In [ ]:
# Usar métodos do Pandas
print("Estatísticas descritivas:")
print(df1.describe())

print("\nInformações do DataFrame:")
print(df1.info())

## 6. Exportação para CSV

Salvando resultados de queries em arquivos CSV.

### 6.1 Exportação Básica

In [ ]:
# Criar uma conexão
con = duckdb.connect()

# Criar tabela temporária
con.sql("""
    CREATE TABLE funcionarios AS 
    SELECT * FROM 'example.csv'
""")

# Exportar resultado de query
result = con.sql("SELECT * FROM funcionarios WHERE salario > 6000")
result.write_csv("salarios_altos.csv")

print("✓ Arquivo 'salarios_altos.csv' criado!")
print(f"Tamanho: {os.path.getsize('salarios_altos.csv')} bytes")

# Verificar conteúdo
print("\nConteúdo do arquivo exportado:")
duckdb.sql("SELECT * FROM 'salarios_altos.csv'").show()

### 6.2 Exportação com COPY TO

In [ ]:
# Usar COPY TO (mais controle)
con.sql("""
    COPY (
        SELECT nome, idade, cidade
        FROM funcionarios
        ORDER BY idade
    ) TO 'resumo_funcionarios.csv' (HEADER, DELIMITER ',')
""")

print("✓ Arquivo 'resumo_funcionarios.csv' criado com COPY TO!")

# Verificar
print("\nConteúdo:")
duckdb.sql("SELECT * FROM 'resumo_funcionarios.csv'").show()

## 7. Configurações de Leitura

Personalizando como o DuckDB lê arquivos CSV.

### 7.1 Delimitador Personalizado

In [ ]:
# Ler CSV com ponto e vírgula
print("Lendo CSV com delimitador ';':")
produtos = duckdb.read_csv("produtos.csv", sep=";")
produtos.show()

print("\n✓ CSV com delimitador customizado lido!")

### 7.2 CSV sem Header

In [ ]:
# Criar CSV sem header
csv_sem_header = """1,Produto A,100.00
2,Produto B,200.00
3,Produto C,150.00
"""

with open("sem_header.csv", "w") as f:
    f.write(csv_sem_header)

# Ler especificando nomes de colunas
print("CSV sem header:")
duckdb.sql("""
    SELECT * FROM read_csv(
        'sem_header.csv',
        header=false,
        columns={'id': 'INTEGER', 'nome': 'VARCHAR', 'preco': 'DECIMAL'}
    )
""").show()

print("\n✓ CSV sem header lido com schema personalizado!")

### 7.3 Tipos de Dados Específicos

In [ ]:
# Forçar tipos específicos
print("Forçando tipos de dados:")
duckdb.sql("""
    SELECT * FROM read_csv(
        'example.csv',
        columns={
            'id': 'VARCHAR',        -- ID como string
            'nome': 'VARCHAR',
            'idade': 'INTEGER',
            'cidade': 'VARCHAR',
            'salario': 'DECIMAL(10,2)'
        }
    )
    LIMIT 3
""").show()

print("\n✓ Tipos forçados aplicados!")

## 8. Lidando com Arquivos Grandes

DuckDB é otimizado para grandes volumes de dados.

### 8.1 Criar Arquivo Grande de Teste

In [ ]:
# Criar CSV com muitas linhas
con.sql("""
    COPY (
        SELECT 
            i AS id,
            'Usuario_' || i AS nome,
            20 + (i % 50) AS idade,
            (i % 10 + 1) * 1000.0 AS salario
        FROM range(1, 100001) t(i)
    ) TO 'grande.csv' (HEADER)
""")

tamanho = os.path.getsize('grande.csv')
print(f"✓ Arquivo 'grande.csv' criado!")
print(f"Tamanho: {tamanho:,} bytes ({tamanho/1024/1024:.2f} MB)")
print(f"Registros: 100,000")

### 8.2 Query Eficiente em Arquivo Grande

In [ ]:
import time

# Consulta rápida
start = time.time()

resultado = duckdb.sql("""
    SELECT 
        COUNT(*) AS total,
        AVG(idade) AS idade_media,
        AVG(salario) AS salario_medio,
        MAX(salario) AS max_salario
    FROM 'grande.csv'
""")

end = time.time()

print("Estatísticas do arquivo grande:")
resultado.show()

print(f"\n⚡ Tempo de execução: {(end-start)*1000:.2f}ms")
print("✓ DuckDB é muito rápido com arquivos grandes!")

## 9. Compressão (CSV.GZ)

DuckDB suporta leitura e escrita de arquivos comprimidos.

### 9.1 Exportar CSV Comprimido

In [ ]:
# Exportar como CSV.GZ
con.sql("""
    COPY (
        SELECT * FROM 'example.csv'
    ) TO 'example_compressed.csv.gz' (HEADER, COMPRESSION GZIP)
""")

# Comparar tamanhos
tamanho_original = os.path.getsize('example.csv')
tamanho_comprimido = os.path.getsize('example_compressed.csv.gz')

print("Comparação de tamanhos:")
print(f"Original:    {tamanho_original:,} bytes")
print(f"Comprimido:  {tamanho_comprimido:,} bytes")
print(f"Economia:    {(1 - tamanho_comprimido/tamanho_original)*100:.1f}%")

### 9.2 Ler CSV Comprimido

In [ ]:
# Ler arquivo .gz diretamente (automático!)
print("Lendo arquivo comprimido:")
duckdb.sql("SELECT * FROM 'example_compressed.csv.gz' LIMIT 5").show()

print("\n✓ DuckDB detecta compressão automaticamente!")

## 10. Múltiplos Arquivos (Glob Patterns)

Ler vários arquivos CSV de uma vez.

### 10.1 Criar Múltiplos Arquivos

In [ ]:
# Criar diretório para os arquivos
os.makedirs('vendas', exist_ok=True)

# Criar 3 arquivos de vendas
for mes in ['janeiro', 'fevereiro', 'marco']:
    con.sql(f"""
        COPY (
            SELECT 
                i AS id,
                '{mes}' AS mes,
                'Produto_' || (i % 10) AS produto,
                (i % 100 + 50) * 10.0 AS valor
            FROM range(1, 101) t(i)
        ) TO 'vendas/vendas_{mes}.csv' (HEADER)
    """)

print("✓ Arquivos criados:")
for arquivo in os.listdir('vendas'):
    print(f"  - vendas/{arquivo}")

### 10.2 Ler Todos os Arquivos de Uma Vez

In [ ]:
# Usar glob pattern para ler todos os CSVs
print("Lendo todos os arquivos com glob pattern:")
duckdb.sql("""
    SELECT 
        mes,
        COUNT(*) AS total_vendas,
        SUM(valor) AS valor_total,
        AVG(valor) AS valor_medio
    FROM 'vendas/*.csv'
    GROUP BY mes
    ORDER BY mes
""").show()

print("\n✓ Todos os arquivos processados em uma query!")

## 11. Criação de Tabelas Persistentes

Importar CSV para tabelas permanentes.

### 11.1 Criar Tabela a Partir de CSV

In [ ]:
# Criar banco de dados persistente
con_persist = duckdb.connect('meu_banco.db')

# Criar tabela a partir do CSV
con_persist.sql("""
    CREATE TABLE pessoas AS 
    SELECT * FROM 'example.csv'
""")

print("✓ Tabela 'pessoas' criada no banco de dados!")

# Verificar
print("\nDados na tabela:")
con_persist.sql("SELECT * FROM pessoas LIMIT 5").show()

# Listar tabelas
print("\nTabelas no banco:")
con_persist.sql("SHOW TABLES").show()

con_persist.close()

## 12. Performance e Boas Práticas

### 📊 Dicas de Performance:

1. **Use SQL Diretamente**: `SELECT * FROM 'file.csv'` é mais rápido que carregar para memória
2. **Filtre Cedo**: Aplique WHERE antes de converter para DataFrame
3. **Use Glob Patterns**: Processe múltiplos arquivos em uma query
4. **Comprima**: Use .gz para economizar espaço (leitura é automática)
5. **Especifique Tipos**: Define tipos manualmente para melhor performance
6. **Evite SELECT ***: Selecione apenas colunas necessárias

### 12.1 Exemplo de Query Otimizada

In [ ]:
# ❌ Não Otimizado: Carrega tudo, depois filtra
# df = duckdb.read_csv('grande.csv').df()
# df_filtrado = df[df['salario'] > 5000]

# ✅ Otimizado: Filtra antes de carregar
df_otimizado = duckdb.sql("""
    SELECT id, nome, salario
    FROM 'grande.csv'
    WHERE salario > 5000
    LIMIT 10
""").df()

print("DataFrame otimizado (filtrado na origem):")
print(df_otimizado)
print(f"\nShape: {df_otimizado.shape}")
print("\n✓ Muito mais eficiente!")

### 12.2 Benchmark Comparativo

In [ ]:
import time

# Teste 1: DuckDB direto
start = time.time()
result_duckdb = duckdb.sql("""
    SELECT COUNT(*), AVG(salario)
    FROM 'grande.csv'
    WHERE idade > 30
""").fetchall()
time_duckdb = time.time() - start

# Teste 2: Pandas tradicional
start = time.time()
df_pandas = pd.read_csv('grande.csv')
result_pandas = df_pandas[df_pandas['idade'] > 30]['salario'].agg(['count', 'mean'])
time_pandas = time.time() - start

print("Benchmark: Agregação em 100k linhas")
print("=" * 40)
print(f"DuckDB:  {time_duckdb*1000:.2f}ms")
print(f"Pandas:  {time_pandas*1000:.2f}ms")
print(f"\nDuckDB é {time_pandas/time_duckdb:.1f}x mais rápido!")

## 🎯 Resumo do Capítulo

Neste capítulo, exploramos:

1. ✅ **Leitura básica** com `read_csv()` e SQL direto
2. ✅ **Queries SQL** diretamente em arquivos CSV
3. ✅ **Conversão** para Pandas DataFrame
4. ✅ **Exportação** para CSV com múltiplas opções
5. ✅ **Configurações** personalizadas (delimitadores, headers, tipos)
6. ✅ **Performance** com arquivos grandes
7. ✅ **Compressão** automática (.gz)
8. ✅ **Glob patterns** para múltiplos arquivos
9. ✅ **Tabelas persistentes** em banco de dados
10. ✅ **Boas práticas** e otimizações

### 🔑 Pontos-Chave:
- DuckDB lê CSV **nativamente** com SQL
- **Extremamente rápido** para grandes volumes
- **Integração perfeita** com Pandas
- Suporta **compressão** transparente
- **Glob patterns** para múltiplos arquivos

### 📚 Próximo Capítulo:
Trabalhando com Parquet!

## 🧹 Limpeza (Opcional)

In [ ]:
import shutil

# Listar arquivos criados
arquivos_para_remover = [
    'example.csv', 'produtos.csv', 'sem_header.csv',
    'salarios_altos.csv', 'resumo_funcionarios.csv',
    'grande.csv', 'example_compressed.csv.gz',
    'meu_banco.db'
]

for arquivo in arquivos_para_remover:
    if os.path.exists(arquivo):
        os.remove(arquivo)
        print(f"✓ Removido: {arquivo}")

# Remover diretório
if os.path.exists('vendas'):
    shutil.rmtree('vendas')
    print("✓ Diretório 'vendas' removido")

print("\n✓ Limpeza concluída!")

# Capítulo 03 - Importação e Exportação de CSV

Este notebook explora todas as formas de trabalhar com arquivos CSV no DuckDB, desde leitura simples até operações avançadas com configurações personalizadas.

## 📚 Tópicos Abordados:
1. Instalação e Preparação
2. Criando Dados de Exemplo
3. Leitura Básica de CSV
4. Leitura com Query SQL Direta
5. Conversão para DataFrame (Pandas)
6. Exportação para CSV
7. Configurações de Leitura (Delimitadores, Headers)
8. Lidando com Arquivos Grandes
9. Compressão (CSV.GZ)
10. Múltiplos Arquivos (Glob Patterns)
11. Criação de Tabelas Persistentes
12. Performance e Boas Práticas

## 1. Instalação e Preparação

Instalando dependências necessárias.

In [ ]:
# Instalar DuckDB e Pandas
%pip install duckdb pandas -q
print("✓ Pacotes instalados!")

In [ ]:
import duckdb
import pandas as pd
import os

print(f"DuckDB versão: {duckdb.__version__}")
print(f"Pandas versão: {pd.__version__}")
print("\n✓ Imports realizados com sucesso!")

## 2. Criando Dados de Exemplo

Vamos criar alguns arquivos CSV de exemplo para trabalhar.

In [ ]:
# Criar arquivo CSV simples
csv_content = """id,nome,idade,cidade,salario
1,Alice,28,São Paulo,5500.00
2,Bob,35,Rio de Janeiro,7200.00
3,Charlie,42,Belo Horizonte,6800.00
4,Diana,31,Curitiba,5900.00
5,Eduardo,29,Porto Alegre,6100.00
6,Fernanda,38,Salvador,7500.00
7,Gabriel,45,Brasília,8200.00
8,Helena,33,Fortaleza,6300.00
"""

with open("example.csv", "w", encoding="utf-8") as f:
    f.write(csv_content)

print("✓ Arquivo 'example.csv' criado!")
print(f"Tamanho: {os.path.getsize('example.csv')} bytes")

In [ ]:
# Criar arquivo CSV com delimitador diferente (ponto e vírgula)
csv_semicolon = """produto;categoria;preco;estoque
Notebook;Eletrônicos;3500.00;15
Mouse;Eletrônicos;50.00;120
Teclado;Eletrônicos;150.00;80
Cadeira;Móveis;899.00;25
Mesa;Móveis;1200.00;10
"""

with open("produtos.csv", "w", encoding="utf-8") as f:
    f.write(csv_semicolon)

print("✓ Arquivo 'produtos.csv' criado (delimitador: ';')!")

## 3. Leitura Básica de CSV

DuckDB oferece formas simples de ler arquivos CSV.

### 3.1 Usando `read_csv()`

In [ ]:
import duckdb

# Ler CSV como Relation (objeto DuckDB)
rel = duckdb.read_csv("example.csv")

print("Tipo do objeto:", type(rel))
print("\nPrimeiras linhas:")
rel.show()

print("\n✓ CSV lido com sucesso!")

### 3.2 Inspecionar Schema

In [ ]:
# Ver estrutura do CSV
print("Schema detectado:")
print(rel)

# Ou usando SQL
print("\nUsando DESCRIBE:")
duckdb.sql("DESCRIBE SELECT * FROM 'example.csv'").show()

## 4. Leitura com Query SQL Direta

DuckDB permite consultar CSV diretamente com SQL!

### 4.1 Query Simples

In [ ]:
# Query direta no arquivo CSV
print("Consulta SQL direta no CSV:")
result = duckdb.sql("SELECT * FROM 'example.csv'")
result.show()

print("\n✓ Query executada diretamente no arquivo!")

### 4.2 Queries com Filtros

In [ ]:
# Filtrar dados
print("Pessoas com idade > 35:")
duckdb.sql("""
    SELECT nome, idade, cidade, salario
    FROM 'example.csv'
    WHERE idade > 35
    ORDER BY idade DESC
""").show()

print("\nSalários acima de 7000:")
duckdb.sql("""
    SELECT nome, cidade, salario
    FROM 'example.csv'
    WHERE salario >= 7000
""").show()

### 4.3 Agregações

In [ ]:
# Estatísticas
print("Estatísticas gerais:")
duckdb.sql("""
    SELECT 
        COUNT(*) AS total_pessoas,
        AVG(idade) AS idade_media,
        AVG(salario) AS salario_medio,
        MAX(salario) AS maior_salario,
        MIN(salario) AS menor_salario
    FROM 'example.csv'
""").show()

print("\nPessoas por cidade:")
duckdb.sql("""
    SELECT 
        cidade,
        COUNT(*) AS quantidade,
        AVG(salario) AS salario_medio
    FROM 'example.csv'
    GROUP BY cidade
    ORDER BY salario_medio DESC
""").show()

## 5. Conversão para DataFrame (Pandas)

Integração perfeita com Pandas para análise de dados.

### 5.1 Converter para Pandas

In [ ]:
# Método 1: Via read_csv + df()
df1 = duckdb.read_csv("example.csv").df()

print("DataFrame via read_csv:")
print(df1)
print(f"\nTipo: {type(df1)}")
print(f"Shape: {df1.shape}")

In [ ]:
# Método 2: Via SQL + df()
df2 = duckdb.sql("SELECT * FROM 'example.csv' WHERE idade > 30").df()

print("DataFrame via SQL (com filtro):")
print(df2)
print(f"\nShape: {df2.shape}")

### 5.2 Análise com Pandas

In [ ]:
# Usar métodos do Pandas
print("Estatísticas descritivas:")
print(df1.describe())

print("\nInformações do DataFrame:")
print(df1.info())

## 6. Exportação para CSV

Salvando resultados de queries em arquivos CSV.

### 6.1 Exportação Básica

In [ ]:
# Criar uma conexão
con = duckdb.connect()

# Criar tabela temporária
con.sql("""
    CREATE TABLE funcionarios AS 
    SELECT * FROM 'example.csv'
""")

# Exportar resultado de query
result = con.sql("SELECT * FROM funcionarios WHERE salario > 6000")
result.write_csv("salarios_altos.csv")

print("✓ Arquivo 'salarios_altos.csv' criado!")
print(f"Tamanho: {os.path.getsize('salarios_altos.csv')} bytes")

# Verificar conteúdo
print("\nConteúdo do arquivo exportado:")
duckdb.sql("SELECT * FROM 'salarios_altos.csv'").show()

### 6.2 Exportação com COPY TO

In [ ]:
# Usar COPY TO (mais controle)
con.sql("""
    COPY (
        SELECT nome, idade, cidade
        FROM funcionarios
        ORDER BY idade
    ) TO 'resumo_funcionarios.csv' (HEADER, DELIMITER ',')
""")

print("✓ Arquivo 'resumo_funcionarios.csv' criado com COPY TO!")

# Verificar
print("\nConteúdo:")
duckdb.sql("SELECT * FROM 'resumo_funcionarios.csv'").show()

## 7. Configurações de Leitura

Personalizando como o DuckDB lê arquivos CSV.

### 7.1 Delimitador Personalizado

In [ ]:
# Ler CSV com ponto e vírgula
print("Lendo CSV com delimitador ';':")
produtos = duckdb.read_csv("produtos.csv", sep=";")
produtos.show()

print("\n✓ CSV com delimitador customizado lido!")

### 7.2 CSV sem Header

In [ ]:
# Criar CSV sem header
csv_sem_header = """1,Produto A,100.00
2,Produto B,200.00
3,Produto C,150.00
"""

with open("sem_header.csv", "w") as f:
    f.write(csv_sem_header)

# Ler especificando nomes de colunas
print("CSV sem header:")
duckdb.sql("""
    SELECT * FROM read_csv(
        'sem_header.csv',
        header=false,
        columns={'id': 'INTEGER', 'nome': 'VARCHAR', 'preco': 'DECIMAL'}
    )
""").show()

print("\n✓ CSV sem header lido com schema personalizado!")

### 7.3 Tipos de Dados Específicos

In [ ]:
# Forçar tipos específicos
print("Forçando tipos de dados:")
duckdb.sql("""
    SELECT * FROM read_csv(
        'example.csv',
        columns={
            'id': 'VARCHAR',        -- ID como string
            'nome': 'VARCHAR',
            'idade': 'INTEGER',
            'cidade': 'VARCHAR',
            'salario': 'DECIMAL(10,2)'
        }
    )
    LIMIT 3
""").show()

print("\n✓ Tipos forçados aplicados!")

## 8. Lidando com Arquivos Grandes

DuckDB é otimizado para grandes volumes de dados.

### 8.1 Criar Arquivo Grande de Teste

In [ ]:
# Criar CSV com muitas linhas
con.sql("""
    COPY (
        SELECT 
            i AS id,
            'Usuario_' || i AS nome,
            20 + (i % 50) AS idade,
            (i % 10 + 1) * 1000.0 AS salario
        FROM range(1, 100001) t(i)
    ) TO 'grande.csv' (HEADER)
""")

tamanho = os.path.getsize('grande.csv')
print(f"✓ Arquivo 'grande.csv' criado!")
print(f"Tamanho: {tamanho:,} bytes ({tamanho/1024/1024:.2f} MB)")
print(f"Registros: 100,000")

### 8.2 Query Eficiente em Arquivo Grande

In [ ]:
import time

# Consulta rápida
start = time.time()

resultado = duckdb.sql("""
    SELECT 
        COUNT(*) AS total,
        AVG(idade) AS idade_media,
        AVG(salario) AS salario_medio,
        MAX(salario) AS max_salario
    FROM 'grande.csv'
""")

end = time.time()

print("Estatísticas do arquivo grande:")
resultado.show()

print(f"\n⚡ Tempo de execução: {(end-start)*1000:.2f}ms")
print("✓ DuckDB é muito rápido com arquivos grandes!")

## 9. Compressão (CSV.GZ)

DuckDB suporta leitura e escrita de arquivos comprimidos.

### 9.1 Exportar CSV Comprimido

In [ ]:
# Exportar como CSV.GZ
con.sql("""
    COPY (
        SELECT * FROM 'example.csv'
    ) TO 'example_compressed.csv.gz' (HEADER, COMPRESSION GZIP)
""")

# Comparar tamanhos
tamanho_original = os.path.getsize('example.csv')
tamanho_comprimido = os.path.getsize('example_compressed.csv.gz')

print("Comparação de tamanhos:")
print(f"Original:    {tamanho_original:,} bytes")
print(f"Comprimido:  {tamanho_comprimido:,} bytes")
print(f"Economia:    {(1 - tamanho_comprimido/tamanho_original)*100:.1f}%")

### 9.2 Ler CSV Comprimido

In [ ]:
# Ler arquivo .gz diretamente (automático!)
print("Lendo arquivo comprimido:")
duckdb.sql("SELECT * FROM 'example_compressed.csv.gz' LIMIT 5").show()

print("\n✓ DuckDB detecta compressão automaticamente!")

## 10. Múltiplos Arquivos (Glob Patterns)

Ler vários arquivos CSV de uma vez.

### 10.1 Criar Múltiplos Arquivos

In [ ]:
# Criar diretório para os arquivos
os.makedirs('vendas', exist_ok=True)

# Criar 3 arquivos de vendas
for mes in ['janeiro', 'fevereiro', 'marco']:
    con.sql(f"""
        COPY (
            SELECT 
                i AS id,
                '{mes}' AS mes,
                'Produto_' || (i % 10) AS produto,
                (i % 100 + 50) * 10.0 AS valor
            FROM range(1, 101) t(i)
        ) TO 'vendas/vendas_{mes}.csv' (HEADER)
    """)

print("✓ Arquivos criados:")
for arquivo in os.listdir('vendas'):
    print(f"  - vendas/{arquivo}")

### 10.2 Ler Todos os Arquivos de Uma Vez

In [ ]:
# Usar glob pattern para ler todos os CSVs
print("Lendo todos os arquivos com glob pattern:")
duckdb.sql("""
    SELECT 
        mes,
        COUNT(*) AS total_vendas,
        SUM(valor) AS valor_total,
        AVG(valor) AS valor_medio
    FROM 'vendas/*.csv'
    GROUP BY mes
    ORDER BY mes
""").show()

print("\n✓ Todos os arquivos processados em uma query!")

## 11. Criação de Tabelas Persistentes

Importar CSV para tabelas permanentes.

### 11.1 Criar Tabela a Partir de CSV

In [ ]:
# Criar banco de dados persistente
con_persist = duckdb.connect('meu_banco.db')

# Criar tabela a partir do CSV
con_persist.sql("""
    CREATE TABLE pessoas AS 
    SELECT * FROM 'example.csv'
""")

print("✓ Tabela 'pessoas' criada no banco de dados!")

# Verificar
print("\nDados na tabela:")
con_persist.sql("SELECT * FROM pessoas LIMIT 5").show()

# Listar tabelas
print("\nTabelas no banco:")
con_persist.sql("SHOW TABLES").show()

con_persist.close()

## 12. Performance e Boas Práticas

### 📊 Dicas de Performance:

1. **Use SQL Diretamente**: `SELECT * FROM 'file.csv'` é mais rápido que carregar para memória
2. **Filtre Cedo**: Aplique WHERE antes de converter para DataFrame
3. **Use Glob Patterns**: Processe múltiplos arquivos em uma query
4. **Comprima**: Use .gz para economizar espaço (leitura é automática)
5. **Especifique Tipos**: Define tipos manualmente para melhor performance
6. **Evite SELECT ***: Selecione apenas colunas necessárias

### 12.1 Exemplo de Query Otimizada

In [ ]:
# ❌ Não Otimizado: Carrega tudo, depois filtra
# df = duckdb.read_csv('grande.csv').df()
# df_filtrado = df[df['salario'] > 5000]

# ✅ Otimizado: Filtra antes de carregar
df_otimizado = duckdb.sql("""
    SELECT id, nome, salario
    FROM 'grande.csv'
    WHERE salario > 5000
    LIMIT 10
""").df()

print("DataFrame otimizado (filtrado na origem):")
print(df_otimizado)
print(f"\nShape: {df_otimizado.shape}")
print("\n✓ Muito mais eficiente!")

### 12.2 Benchmark Comparativo

In [ ]:
import time

# Teste 1: DuckDB direto
start = time.time()
result_duckdb = duckdb.sql("""
    SELECT COUNT(*), AVG(salario)
    FROM 'grande.csv'
    WHERE idade > 30
""").fetchall()
time_duckdb = time.time() - start

# Teste 2: Pandas tradicional
start = time.time()
df_pandas = pd.read_csv('grande.csv')
result_pandas = df_pandas[df_pandas['idade'] > 30]['salario'].agg(['count', 'mean'])
time_pandas = time.time() - start

print("Benchmark: Agregação em 100k linhas")
print("=" * 40)
print(f"DuckDB:  {time_duckdb*1000:.2f}ms")
print(f"Pandas:  {time_pandas*1000:.2f}ms")
print(f"\nDuckDB é {time_pandas/time_duckdb:.1f}x mais rápido!")

## 🎯 Resumo do Capítulo

Neste capítulo, exploramos:

1. ✅ **Leitura básica** com `read_csv()` e SQL direto
2. ✅ **Queries SQL** diretamente em arquivos CSV
3. ✅ **Conversão** para Pandas DataFrame
4. ✅ **Exportação** para CSV com múltiplas opções
5. ✅ **Configurações** personalizadas (delimitadores, headers, tipos)
6. ✅ **Performance** com arquivos grandes
7. ✅ **Compressão** automática (.gz)
8. ✅ **Glob patterns** para múltiplos arquivos
9. ✅ **Tabelas persistentes** em banco de dados
10. ✅ **Boas práticas** e otimizações

### 🔑 Pontos-Chave:
- DuckDB lê CSV **nativamente** com SQL
- **Extremamente rápido** para grandes volumes
- **Integração perfeita** com Pandas
- Suporta **compressão** transparente
- **Glob patterns** para múltiplos arquivos

### 📚 Próximo Capítulo:
Trabalhando com Parquet!

## 🧹 Limpeza (Opcional)

In [ ]:
import shutil

# Listar arquivos criados
arquivos_para_remover = [
    'example.csv', 'produtos.csv', 'sem_header.csv',
    'salarios_altos.csv', 'resumo_funcionarios.csv',
    'grande.csv', 'example_compressed.csv.gz',
    'meu_banco.db'
]

for arquivo in arquivos_para_remover:
    if os.path.exists(arquivo):
        os.remove(arquivo)
        print(f"✓ Removido: {arquivo}")

# Remover diretório
if os.path.exists('vendas'):
    shutil.rmtree('vendas')
    print("✓ Diretório 'vendas' removido")

print("\n✓ Limpeza concluída!")

# Capitulo 03 Importacao Exportacao Csv

Notebook gerado automaticamente a partir do código fonte python.


In [ ]:
# -*- coding: utf-8 -*-
"""
capitulo_03_importacao_exportacao_csv
"""

# capitulo_03_importacao_exportacao_csv
import duckdb
import os

In [ ]:
import duckdb

# Executar query e salvar como CSV
result = duckdb.sql("SELECT * FROM 'c:/projetos/learn-apps-code/sales_data.csv'")
result.write_csv("out.csv")

# Com método direto
duckdb.sql("SELECT * FROM 'c:/projetos/learn-apps-code/sales_data.csv'").write_csv("out2.csv")
print("CSV files exported successfully")

## 5. Exportação para CSV

Salvando resultados de queries em arquivos CSV.

In [ ]:
import duckdb

# Ler e converter para DataFrame
df = duckdb.read_csv("c:/projetos/learn-apps-code/sales_data.csv").df()

# Ou diretamente
df = duckdb.sql("SELECT * FROM 'c:/projetos/learn-apps-code/sales_data.csv'").df()
print(df)

## 4. Conversão para DataFrame

Convertendo CSV diretamente para Pandas DataFrame.

In [ ]:
import duckdb

# Ler CSV como Relation
rel = duckdb.read_csv("c:/projetos/learn-apps-code/sales_data.csv")
print(rel)

# Query direta
result = duckdb.sql("SELECT * FROM 'c:/projetos/learn-apps-code/sales_data.csv'")
result.show()

## 3. Leitura de Arquivos CSV

Diferentes formas de ler arquivos CSV com DuckDB.

## 2. Imports

In [ ]:
%pip install duckdb -q

## 1. Instalação de Dependências